In [ ]:
from azure.mgmt.compute import ComputeManagementClient
from azure.mgmt.network import NetworkManagementClient
from azure.identity import DefaultAzureCredential

In [ ]:
# ID de tu suscripción
subscription_id = ""
resource_group = ""
vm_name = ""


# Autenticación
credential = DefaultAzureCredential()

In [ ]:
from func.funciones_vm import eliminar_vm
from auth.autenticacion import ClienteAzure
from azure.identity import DefaultAzureCredential

credential = DefaultAzureCredential()
subscription_id = ""

cliente_azure = ClienteAzure(credential,subscription_id)
resource_group = ""

eliminar_vm(
    cliente_azure,
    resource_group,
    'test-cluster-worker-2'
)

In [ ]:
# Cliente de Compute
compute_client = ComputeManagementClient(credential, subscription_id)

# Cliente de Red
network_client = NetworkManagementClient(credential, subscription_id)

In [ ]:
# Obtener la VM (antes de borrarla)
vm = compute_client.virtual_machines.get(resource_group, vm_name)

In [ ]:
# Nombre del disco del sistema operativo
os_disk_name = vm.storage_profile.os_disk.name

In [ ]:
vm.storage_profile.os_disk.id

In [ ]:
nic_id = vm.network_profile.network_interfaces[0].id
nic_name = nic_id.split("/")[-1]

In [ ]:
# NIC asociada
nic = network_client.network_interfaces.get(resource_group, nic_name)
ip_config = nic.ip_configurations[0]
public_ip_id = ip_config.public_ip_address.id if ip_config.public_ip_address else None
public_ip_name = public_ip_id.split("/")[-1] if public_ip_id else None

In [ ]:
# Eliminar la VM
compute_client.virtual_machines.begin_delete(resource_group, vm_name).result()
print(f"VM {vm_name} eliminada.")

In [ ]:
# Eliminar NIC
network_client.network_interfaces.begin_delete(resource_group, nic_name).result()
print(f"NIC {nic_name} eliminada.")

In [ ]:
# Eliminar disco
compute_client.disks.begin_delete(resource_group, os_disk_name).result()
print(f"Disco {os_disk_name} eliminado.")


In [ ]:
# 6. Eliminar la IP pública (si tenía)
if public_ip_name:
    network_client.public_ip_addresses.begin_delete(resource_group, public_ip_name).result()
    print(f"✅ IP pública {public_ip_name} eliminada.")

In [ ]:
if nic.network_security_group:
    nsg_id = nic.network_security_group.id
    nsg_name = nsg_id.split('/')[-1]
    print(f"La VM '{vm_name}' tiene asignado el NSG: {nsg_name}")

    network_client.network_security_groups.begin_delete(resource_group, nsg_name).result()
    print(f"NSG '{nsg_name}' eliminado.")
else:
    print(f"La VM '{vm_name}' no tiene NSG asignado")
